# Imports

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

# Problem 1 : 
**Given an unknown-lenght array of the form 1, 1, 1, ..., 1, 0, 0, ..., 0, ... (with 1s at
the first places and 0s after this), and the interface function getElement(array A, int i)
that returns the i-th element of the given array, describe an efficient algorithm to count the
number of 1s.**

We will try to solve this problem with dichotomy. 

We can solve this problem by itering over the whole array and increment an integer. The time complexity of this simple solution  would be O(n). 

We can use dichotomy to find the solution in O(Logn) time. The idea is to find the last occurrence of 1 using Dichotomy : 
1. We split the array in two, getting the mid element. 
2. We test wether the mid element is the last 1 in the array. In that case, we return the mid index.
3. If not and mid index element is 0, then we split the array and two and do the same on the left side of the array, else the right side.

In [83]:
array = [1,1,1,1,1,1,1,1,0,0,0,0]

def getElement(array, i):
    return array[i]

# Returns number of 1's in array[low..high].
def number_ones(array, low, high):
    #If high < low then we have no 1s in the array
    if high >= low:
        
        # get the middle index
        mid = low + (high-low)//2
         
        # check if the element at mid index is 1
        if getElement(array, mid)==1 :
            #Check if the element at mid+1 is 0 or mid == high, if that's the case, return the index of mid
            if (mid == high or getElement(array, mid+1)==0) :
                return mid+1
            
            # If the element at mid index is not the last 1, we split the array and recur from the right side
            else : 
                return number_ones(array, (mid+1), high)
            
        # else we recur from the left side
        return number_ones(array, low, mid-1)
     
    return 0

print("Number of 1s in the array :", number_ones(array, 0, len(array)-1))

Number of 1s in the array : 8


# Problem 2 : 
**Given a set of all integers from 1 to N missing only one of them, identify which
one is missing in an efficient way.**

In the same way, we can use dichotomy to get a O(logn) time complexity. We will try to find the first index for which getElement(index) != index 

In [160]:
array = list(np.arange(0,20))
del array[8]
print("array with missing 8 : ")
print(array)
print("indexes :")
print(list(np.arange(19)))

array with missing 8 : 
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
indexes :
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [158]:
# o(logn) complexity function that returns the missing element of a 1..N array. It uses Dichotomy and recursivity 
def missing_elt(array, low, high): 
    if high >= low:
        mid = low + (high-low)//2
        #check if mid element is different from mid index
        if array[mid] != mid:
            
            #we check if it's the first one in the array, in that case, we return mid index
            if array[mid-1] == mid-1 or mid==low:
                return mid
            else :
            #Otherwise we recur from the left
                return missing_elt(array, low, mid-1)
        else:
            return missing_elt(array, mid+1, high)
    else: 
        return 'no missing element !'
    
    
print("The missing element in the array is :", missing_elt(array, 0, len(array)-1))

The missing element in the array is : 5


# Problem 5 : 
** In Euroleague basketball competition the Performance Index Rating (PIR) is
used to determine the MVP of each round. It is a statistical formula that uses the players’
stats during each game, e.g. points, rebounds, assists, etc. The stats can be found in the
GAME CENTER section of http://www.euroleague.net for the majority of the games in the
last years.**


**Assuming that we don’t know the actual PIR formula, what is your approach(es) in order
to determine it? Please, give as many details as possible. **

**For your consideration, the answer can be found here and the actual PIR formula is
(Points + Rebounds + Assists + Steals + Blocks + Fouls Drawn) -
(Missed FG + Missed FT + Turnovers + Shots Rejected + Fouls Committed)**


**where FG stands for Field Goals and FT stands for Free Throws. ** 

To solve this, we will first srap the stats and PIR of some players and then we will perform the reverse engineering to find the formula.

## Let's scrap all the stats of some players

We will only scrap the first page as we will have enough data 

In [34]:
#We create a dictionary of indexes used in the website :
dico = {3 : "points", 9 : "rebonds", 10 : "assists", 11 : "steals", 13 : "blocks", 16 : "fouls drown",\
        4 : "missed fg1", 5 : "missed fg2", 6 : "missed ft", 12 : "turnovers", 14 : "shots rejected", 15 : "fouls committed", \
       17 : "PIR"}

#We only scrap the first page
results = requests.get("http://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2017&cat=Valuation&agg=Accumulated")
soup = BeautifulSoup(results.text, 'html.parser')
df = pd.DataFrame(columns=dico.values())

In [89]:
i = 0
#We find all the links leading to specific players pages to get the stats and PIR
for tr in soup.find("table").findAll("tr")[1:]:
    res = requests.get("http://www.euroleague.net/"+tr.findAll("td")[1].find('a', href=True)["href"])
    player = BeautifulSoup(res.text, 'html.parser')
    
    for line in (player.findAll("table")[1].findAll("tr")):
        if "TotalFooter" in str(line) or "AverageFooter" in str(line):
            continue
        td = line.findAll('td')
        try : 
            for key in dico.keys():
                df.loc[i, dico[key]] = td[key].text
            i += 1
        except : 
            pass

Some text replacements and cleaning...

In [90]:
df = df[~df["missed fg1"].str.contains("%")]

In [91]:
df.replace({"\n" : "", "" : "0", '\xa0' : "0"}, regex=True, inplace=True)

In [92]:
#we calculate the number of missed fg and ft 
def parse_str(x):
    temp = x.split("/")
    if len(temp) == 2:
        return int(temp[1]) - int(temp[0])
    else : 
        return(x)
    
df["missed fg1"] = df["missed fg1"].map(lambda x : parse_str(x))
df["missed fg2"] = df["missed fg2"].map(lambda x : parse_str(x))
df["missed ft"] = df["missed ft"].map(lambda x : parse_str(x))


In [93]:
df = df.astype(float)

Now we have clean, good formatted data : 

In [159]:
df.head()

,points,missed fg1,missed fg2,missed ft,rebonds,assists,steals,turnovers,blocks,shots rejected,fouls committed,fouls drown,PIR
0,27.0,2.0,3.0,1.0,4.0,4.0,0.0,2.0,0.0,0.0,1.0,6.0,32.0
1,14.0,1.0,5.0,2.0,6.0,2.0,2.0,2.0,1.0,0.0,2.0,8.0,21.0
2,27.0,4.0,3.0,1.0,8.0,5.0,3.0,1.0,0.0,3.0,0.0,10.0,41.0
3,28.0,2.0,2.0,0.0,9.0,4.0,0.0,3.0,0.0,0.0,2.0,3.0,35.0
4,12.0,3.0,4.0,0.0,7.0,3.0,1.0,3.0,0.0,0.0,1.0,6.0,18.0


### 1st idea :
solve a linar equation

In [94]:
a = df.drop("PIR", axis=1).iloc[:df.shape[1]-1, :]

In [95]:
b = df["PIR"][:df.shape[1]-1]

In [96]:
np.linalg.solve(a, b)

array([ 1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.])

In [97]:
np.dot(df.drop("PIR", axis=1).iloc[:df.shape[1]-1, :], np.linalg.solve(a, b))

array([ 32.,  21.,  41.,  35.,  18.,  30.,  19.,  28.,  14.,  32.,  36.,
        26.])

This gives the right solution. 

This approach could take a while if we had a bigger system and we could have more than 1 solution if we had colinearity between our observations. We can try to approximate the solution.

### 2nd idea : use the least square approximation (we use a linear regression for this) : 

In [98]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(fit_intercept=False)
lr.fit(df.drop("PIR", axis=1), df["PIR"])

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [99]:
lr.coef_


array([ 1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.])

In [106]:
import numpy as np 
print("all equations are verified ? :", all(np.round(np.dot(df.drop("PIR", axis=1), lr.coef_), 1) == df["PIR"].values))

all equations are verified ? : True


We found the right solution but we were lucky that the the equation was linear. 

If it was not the case, we could have tried a regression with polynomial features : sklearn.preprocessing.PolynomialFeatures(). We also could have tried exponential / logarithmic features. 

# Problem 6
**Read the following paper and express your thoughts, ideas, suggestions and/or questions about how the mentioned techniques and applications can be useful to Pricesearcher.**

I believe Text Mining is a central problematic at PriceSearcher as the website compares different string patterns from other commercial websites to get information (prices, reviews, marks...). Managing all the different ways that websites enter their information can be very painful.


Moreover, trying to get all the information manually about all the products (millions I believe) is nearly impossible. The best way is to find an automatic way to scrap, extract and classify the informations. 


Some techniques quoted in the article could be interesting for PriceSearcher :


- **Information Retrieval** : Let's say you want to get information of a given product from a page of a commercial website that you scrapped. The idea is to automatically detect the information we want to get from the page : the price, the name, the reference from a HTML scraped-page. First, we need labelled data : we can do that manually or from a regex based method that we will manually check. Then we can use a word embedding model like **word2vec** to get a  matrix describing each sentence / each word of our document. Finally, we can build a multi-task classification on a Neural Network (an LSTM can do the job or even a Convolutional Neural Network). 


- **Product Matching** : I guess PriceSearcher have a catalog of all the available products. it can be complicated to match products to users' inputs. In that case we can use Fuzzy Search that will match the inputs to the most probable products. 


- **Web Crawler** : The website has a "Reviews" and a "Buying Guides" categories. We must find the corresponding articles by scraping the internet. Then to get the right articles, we need to build a Document Classification.  To do that we can first build features : Tokenization, POS TAG, TFIDF and other features that seem relevant and then build a Supervised Machine Learning algorithm. In case we do not have labelled data, we can scrap a whole different set of articles and perform a Document Clustering. Then, we could find the cluster of articles and then build a supervised model on top of that to gain precision. 
